In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [43]:
# Load dataset
data = pd.read_excel('Dataset.xlsx', engine='openpyxl')
data.head()

,Team,Match Up,Game Date,W/L,MIN,PTS,FGM,FGA,FG%,3PM,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-
0,GSW,GSW vs. PHX,10/24/2023,L,240,104,36,101,35.6,10,...,78.6,18,31,49,19,11,6,11,23,-4
1,PHX,PHX @ GSW,10/24/2023,W,240,108,42,95,44.2,11,...,76.5,17,43,60,23,5,7,19,22,4
2,LAL,LAL @ DEN,10/24/2023,L,240,107,41,90,45.6,10,...,75,13,31,44,23,5,4,12,18,-12
3,DEN,DEN vs. LAL,10/24/2023,W,240,119,48,91,52.7,14,...,75,9,33,42,29,9,6,12,15,12
4,MEM,MEM vs. NOP,10/25/2023,L,240,104,38,91,41.8,12,...,80,8,29,37,23,8,7,13,19,-7


Manipulate Data:

In [44]:
# Convert game date to datetime
data['Game Date'] = pd.to_datetime(data['Game Date'], format='%m/%d/%Y')

# Extract home and guest teams
data['Home Team'] = data['Match Up'].apply(lambda x: x.split(' vs. ')[0] if 'vs.' in x else x.split(' @ ')[1])
data['Guest Team'] = data['Match Up'].apply(lambda x: x.split(' vs. ')[1] if 'vs.' in x else x.split(' @ ')[0])

# Add the correct Label column: 1 if home team won, 0 otherwise
data['Label'] = data.apply(
    lambda row: 1 if ((row['Team'] == row['Home Team']) and (row['W/L'] == 'W')) 
                    or ((row['Team'] == row['Guest Team']) and (row['W/L'] == 'L'))
                else 0,
    axis=1
)
data.head()

,Team,Match Up,Game Date,W/L,MIN,PTS,FGM,FGA,FG%,3PM,...,REB,AST,STL,BLK,TOV,PF,+/-,Home Team,Guest Team,Label
0,GSW,GSW vs. PHX,2023-10-24,L,240,104,36,101,35.6,10,...,49,19,11,6,11,23,-4,GSW,PHX,0
1,PHX,PHX @ GSW,2023-10-24,W,240,108,42,95,44.2,11,...,60,23,5,7,19,22,4,GSW,PHX,0
2,LAL,LAL @ DEN,2023-10-24,L,240,107,41,90,45.6,10,...,44,23,5,4,12,18,-12,DEN,LAL,1
3,DEN,DEN vs. LAL,2023-10-24,W,240,119,48,91,52.7,14,...,42,29,9,6,12,15,12,DEN,LAL,1
4,MEM,MEM vs. NOP,2023-10-25,L,240,104,38,91,41.8,12,...,37,23,8,7,13,19,-7,MEM,NOP,0


Calculating bias for later adding Home Advatange feature:

In [45]:
# Create a binary feature for home games
data['Is_Home'] = data['Match Up'].str.contains(' vs. ').astype(int)

# Prepare features and labels
X = data[['Is_Home']]  # Home/Away indicator
y = (data['W/L'] == 'W').astype(int)  # Convert W/L to binary outcome

# Fit a linear regression model
model = LinearRegression()
model.fit(X, y)

# The coefficient of 'Is_Home' represents the home advantage bias
bias = model.coef_[0]
print(f"Calculated Bias (Regression): {bias}")
new_bias = 1/21 * bias

# Remove the 'Is_Home' column
data = data.drop(columns=['Is_Home'])

Calculated Bias (Regression): 0.08617886178861783


In [ ]:
# dataset with new feature: stability score
# Initialize the new dataset
new_dataset = []

# Iterate over each match-up
for _, row in data.iterrows():
    game_date = row['Game Date']
    home_team = row['Home Team']
    guest_team = row['Guest Team']
    
    # Filter games before the current game date for both teams
    home_team_games = data[(data['Team'] == home_team) & (data['Game Date'] < game_date)]
    guest_team_games = data[(data['Team'] == guest_team) & (data['Game Date'] < game_date)]
    
    # Calculate W/L rates (win rates) for both teams
    home_team_wins = (home_team_games['W/L'] == 'W').sum()
    home_team_total = len(home_team_games)
    guest_team_wins = (guest_team_games['W/L'] == 'W').sum()
    guest_team_total = len(guest_team_games)
    
    home_win_rate = home_team_wins / home_team_total if home_team_total > 0 else 0
    guest_win_rate = guest_team_wins / guest_team_total if guest_team_total > 0 else 0
    
    # Calculate W/L difference
    wl_difference = home_win_rate - guest_win_rate
    
    # Calculate Stability using mean and variance
    if not home_team_games.empty:
        home_mean_performance = home_team_games.iloc[:, 4:].mean(numeric_only=True)
        home_variance_performance = home_team_games.iloc[:, 4:].var(numeric_only=True)
        home_stability = (home_mean_performance / (home_variance_performance + 1e-6)).mean()
        home_stability = min(max(home_stability, -100), 100)
    else:
        home_stability = 0

    if not guest_team_games.empty:
        guest_mean_performance = guest_team_games.iloc[:, 4:].mean(numeric_only=True)
        guest_variance_performance = guest_team_games.iloc[:, 4:].var(numeric_only=True)
        guest_stability = (guest_mean_performance / (guest_variance_performance + 1e-6)).mean()
        guest_stability = min(max(guest_stability, -100), 100)  # Limit stability within [-100, 100]
    else:
        guest_stability = 0

    # Stability difference
    stability = home_stability - guest_stability
    
    # Prepare the new row
    new_row = {
        'Game Date': game_date,
        'Home Team': home_team,
        'Guest Team': guest_team,
        'Label': int(row['Label']),  # Ensure Label is directly copied and kept as integer
        'W/L Difference': wl_difference,
        'Stability': stability
    }
    
    # Add statistics differences (like in the original approach)
    home_team_stats = home_team_games.iloc[:, 4:].mean(numeric_only=True)
    guest_team_stats = guest_team_games.iloc[:, 4:].mean(numeric_only=True)
    
    if not home_team_stats.empty and not guest_team_stats.empty:
        stats_diff = home_team_stats - guest_team_stats
        # Update new_row but ensure no conflict with 'Label'
        new_row.update({k: v for k, v in stats_diff.to_dict().items() if k != 'Label'})
    
    new_dataset.append(new_row)
# Convert to DataFrame
new_dataset_df_1 = pd.DataFrame(new_dataset)

# Save to file for inspection
output_file = 'Dataset_With_Stability.xlsx'
new_dataset_df_1.to_excel(output_file, index=False)

output_file

In [46]:
def standardize_match(row):
    teams = sorted([row['Home Team'], row['Guest Team']])
    standardized_row = {
        'Standard Home Team': teams[0],
        'Standard Guest Team': teams[1],
        'Game Date': row['Game Date'],
        'W/L': 'W' if (row['Home Team'] == teams[0] and row['W/L'] == 'W') or
                       (row['Guest Team'] == teams[0] and row['W/L'] == 'L') else 'L'
    }
    return standardized_row

In [47]:
# Initialize the new dataset
new_dataset = []

# Iterate over each match-up
for _, row in data.iterrows():
    game_date = row['Game Date']
    home_team = row['Home Team']
    guest_team = row['Guest Team']
    
    # Filter games before the current game date for both teams
    home_team_games = data[(data['Team'] == home_team) & (data['Game Date'] < game_date)]
    guest_team_games = data[(data['Team'] == guest_team) & (data['Game Date'] < game_date)]
    
    # Filter previous competitions between home and guest teams
    previous_matches = data[
        ((data['Home Team'] == home_team) & (data['Guest Team'] == guest_team)) |
        ((data['Home Team'] == guest_team) & (data['Guest Team'] == home_team))
    ]
    previous_matches = previous_matches[previous_matches['Game Date'] < game_date]

    standardized_matches = previous_matches.apply(standardize_match, axis=1, result_type='expand')

    standardized_matches = standardized_matches.drop_duplicates(subset=['Game Date', 'Standard Home Team', 'Standard Guest Team'])

    # Calculate Previous Competitions feature (only direct matches between home and guest teams)
    previous_competitions_score = 0
    for _, match in standardized_matches.iterrows():
        if match['Standard Home Team'] == home_team and match['W/L'] == 'W':
            previous_competitions_score += 10
        elif match['Standard Home Team'] == home_team and match['W/L'] == 'L':
            previous_competitions_score -= 10
    
    # Calculate W/L rates (win rates) for both teams
    home_team_wins = (home_team_games['W/L'] == 'W').sum()
    home_team_total = len(home_team_games)
    guest_team_wins = (guest_team_games['W/L'] == 'W').sum()
    guest_team_total = len(guest_team_games)
    
    home_win_rate = home_team_wins / home_team_total if home_team_total > 0 else 0
    guest_win_rate = guest_team_wins / guest_team_total if guest_team_total > 0 else 0
    
    # Calculate W/L difference
    wl_difference = home_win_rate - guest_win_rate
    
    # Calculate Stability using mean and variance
    if not home_team_games.empty:
        home_mean_performance = home_team_games.iloc[:, 4:].mean(numeric_only=True)
        home_variance_performance = home_team_games.iloc[:, 4:].var(numeric_only=True)
        home_stability = (home_mean_performance / (home_variance_performance + 1e-6)).mean()
        home_stability = min(max(home_stability, -100), 100)
    else:
        home_stability = 0

    if not guest_team_games.empty:
        guest_mean_performance = guest_team_games.iloc[:, 4:].mean(numeric_only=True)
        guest_variance_performance = guest_team_games.iloc[:, 4:].var(numeric_only=True)
        guest_stability = (guest_mean_performance / (guest_variance_performance + 1e-6)).mean()
        guest_stability = min(max(guest_stability, -100), 100)  # Limit stability within [-100, 100]
    else:
        guest_stability = 0

    # Stability difference
    stability = home_stability - guest_stability
    
    # Prepare the new row
    new_row = {
        'Game Date': game_date,
        'Home Team': home_team,
        'Guest Team': guest_team,
        'Label': int(row['Label']),  # Ensure Label is directly copied and kept as integer
        'W/L Difference': wl_difference,
        'Stability': stability,
        'Previous Competitions': previous_competitions_score  # Add new feature
    }
    
    # Add statistics differences (like in the original approach)
    home_team_stats = home_team_games.iloc[:, 4:].mean(numeric_only=True)
    guest_team_stats = guest_team_games.iloc[:, 4:].mean(numeric_only=True)
    
    if not home_team_stats.empty and not guest_team_stats.empty:
        stats_diff = home_team_stats - guest_team_stats
        # Update new_row but ensure no conflict with 'Label'
        new_row.update({k: v for k, v in stats_diff.to_dict().items() if k != 'Label'})
    
    new_dataset.append(new_row)

# Convert to DataFrame
new_dataset_df_1 = pd.DataFrame(new_dataset)

# Save to file for inspection
output_file = 'Dataset_with_Stability_and_Previous_Competitions_Corrected.xlsx'
new_dataset_df_1.to_excel(output_file, index=False)

output_file

'Dataset_with_Stability_and_Previous_Competitions_Corrected.xlsx'

In [ ]:
# Initialize the new dataset
new_dataset = []

# Iterate over each match-up
for _, row in data.iterrows():
    game_date = row['Game Date']
    home_team = row['Home Team']
    guest_team = row['Guest Team']
    
    # Filter games before the current game date for both teams
    home_team_games = data[(data['Team'] == home_team) & (data['Game Date'] < game_date)]
    guest_team_games = data[(data['Team'] == guest_team) & (data['Game Date'] < game_date)]
    
    # Calculate W/L rates (win rates) for both teams
    home_team_wins = (home_team_games['W/L'] == 'W').sum()
    home_team_total = len(home_team_games)
    guest_team_wins = (guest_team_games['W/L'] == 'W').sum()
    guest_team_total = len(guest_team_games)
    
    home_win_rate = home_team_wins / home_team_total if home_team_total > 0 else 0
    guest_win_rate = guest_team_wins / guest_team_total if guest_team_total > 0 else 0
    
    # Calculate W/L difference
    wl_difference = home_win_rate - guest_win_rate

    # Determine Home Advantage: 1 for home team, 0 for guest team or neutral field
    home_advantage = 1 if 'vs.' in row['Match Up'] else 0
    
    # Prepare the new row
    new_row = {
        'Game Date': game_date,
        'Home Team': home_team,
        'Guest Team': guest_team,
        'Label': int(row['Label']),  # Ensure Label is directly copied and kept as integer
        'W/L Difference': wl_difference,
        'Home Advantage': home_advantage  # Add the new Home Advantage feature
    }
    
    # Add statistics differences (like in the original approach)
    if not home_team_games.empty and not guest_team_games.empty:
        # Ensure numeric columns only
        home_team_stats = home_team_games.iloc[:, 4:].mean(numeric_only=True)
        guest_team_stats = guest_team_games.iloc[:, 4:].mean(numeric_only=True)
        
        if not home_team_stats.empty and not guest_team_stats.empty:
            stats_diff = home_team_stats - guest_team_stats
            # Update new_row but ensure no conflict with 'Label'
            new_row.update({k: v for k, v in stats_diff.to_dict().items() if k != 'Label'})
    
    new_dataset.append(new_row)

# Convert to DataFrame
new_dataset_df_1 = pd.DataFrame(new_dataset)

# Save to file for inspection
output_file = 'Dataset_with_Home_Advantage_and_Stats.xlsx'
new_dataset_df_1.to_excel(output_file, index=False)

output_file

In [48]:
# Remove rows with NaN values and drop duplicate rows
dataset_df_1 = new_dataset_df_1.dropna().drop_duplicates()

# Save to file for inspection
output_file = 'cleaned_final_dataset_with_features.xlsx'
dataset_df_1.to_excel(output_file, index=False)
output_file

'cleaned_final_dataset_with_features.xlsx'

In [49]:
X = dataset_df_1.drop(columns=['Label', 'Game Date', 'Home Team', 'Guest Team']) # Features: all columns except 'Label'
y = dataset_df_1['Label']  # Labels: the 'Label' column
X.head()

,W/L Difference,Stability,Previous Competitions,MIN,PTS,FGM,FGA,FG%,3PM,3PA,...,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-
55,0.0,0.0,0,-12.5,2.5,2.5,-6.5,5.60,0.5,-9.5,...,-3.0,0.5,7.0,7.5,6.0,-5.0,5.5,5.5,7.0,15.0
57,0.0,0.0,0,0.0,-15.5,-3.5,-5.5,-1.35,-2.5,-2.5,...,-6.5,-3.0,8.0,5.0,-5.0,-0.5,2.0,7.5,-0.5,6.5
68,0.0,0.0,0,0.0,2.5,-2.0,-5.0,0.80,2.0,-3.0,...,3.5,-2.0,6.0,4.0,-2.0,-3.0,-1.0,1.5,1.5,3.5
69,-0.5,0.0,0,12.5,-9.0,0.0,-4.5,0.85,-2.0,-4.5,...,-3.0,-3.5,-3.5,-7.0,-1.0,-6.0,-2.0,3.5,1.5,-19.0
70,0.0,0.0,0,0.0,18.5,5.5,11.0,0.60,8.0,16.5,...,0.0,2.0,-2.0,0.0,3.0,-1.5,0.0,-1.0,0.0,7.5


Using data standardization and L1 regularization to complete Feature Selection.

In [50]:
# Step 1: Standardize the features
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)  # Standardize X to have zero mean and unit variance

# Step 2: Fit Logistic Regression with L1 regularization for binary model
lasso_log_reg = LogisticRegression(penalty='l1', solver='liblinear', C=1.0, random_state=42)  # L1 penalty for feature selection
lasso_log_reg.fit(X_standardized, y)

# Step 3: Identify selected features
selected_features = X.columns[(lasso_log_reg.coef_ != 0).flatten()]  # Keep only features with non-zero coefficients
X_selected = X[selected_features]  # Subset original X with selected features

print(f"Selected features: {list(selected_features)}")
print(X_selected.head())

Selected features: ['W/L Difference', 'Stability', 'MIN', 'FGM', 'FGA', 'FG%', '3PA', '3P%', 'FTM', 'OREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', '+/-']
    W/L Difference  Stability   MIN  FGM   FGA   FG%   3PA   3P%  FTM  OREB  \
55             0.0        0.0 -12.5  2.5  -6.5  5.60  -9.5  9.30 -3.0   0.5   
57             0.0        0.0   0.0 -3.5  -5.5 -1.35  -2.5 -4.40 -6.0  -3.0   
68             0.0        0.0   0.0 -2.0  -5.0  0.80  -3.0  9.85  4.5  -2.0   
69            -0.5        0.0  12.5  0.0  -4.5  0.85  -4.5 -3.40 -7.0  -3.5   
70             0.0        0.0   0.0  5.5  11.0  0.60  16.5  7.80 -0.5   2.0   

    REB  AST  STL  BLK  TOV   PF   +/-  
55  7.5  6.0 -5.0  5.5  5.5  7.0  15.0  
57  5.0 -5.0 -0.5  2.0  7.5 -0.5   6.5  
68  4.0 -2.0 -3.0 -1.0  1.5  1.5   3.5  
69 -7.0 -1.0 -6.0 -2.0  3.5  1.5 -19.0  
70  0.0  3.0 -1.5  0.0 -1.0  0.0   7.5  


Training & Testing using 5-fold cv (including Random Forest, Logistic Regression, Decision Tree, AdaBoost, and QDA):

In [56]:
# Ensure data is shuffled before splitting
X_shuffled, y_shuffled = shuffle(X_selected, y, random_state=1)

# Initialize models
models = {
    'Random Forest': RandomForestClassifier(random_state=0),
    'Logistic Regression': LogisticRegression(random_state=0, max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(random_state=0),
    'AdaBoost': AdaBoostClassifier(random_state=0),
    'QDA': QuadraticDiscriminantAnalysis()
}

# Initialize K-Fold with shuffle
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# Store accuracies for each model
model_accuracies = {model_name: [] for model_name in models}

for train_index, test_index in kf.split(X_shuffled):
    # Split the data into training and testing sets
    X_train, X_test = X_shuffled.iloc[train_index], X_shuffled.iloc[test_index]
    y_train, y_test = y_shuffled.iloc[train_index], y_shuffled.iloc[test_index]
    
    for model_name, model in models.items():
        # Fit the model
        model.fit(X_train, y_train)
        
        # Predict on the test set
        y_pred = model.predict(X_test)
        
        # Calculate accuracy
        accuracy = accuracy_score(y_test, y_pred)
        model_accuracies[model_name].append(accuracy)

# Print results
for model_name, accuracies in model_accuracies.items():
    print(f"{model_name} Accuracy for each fold: {accuracies}")
    print(f"{model_name} Mean accuracy: {sum(accuracies) / len(accuracies):.2f}")

Random Forest Accuracy for each fold: [0.6125, 0.6625, 0.6317991631799164, 0.6276150627615062, 0.6401673640167364]
Random Forest Mean accuracy: 0.63
Logistic Regression Accuracy for each fold: [0.5958333333333333, 0.7125, 0.6694560669456067, 0.6820083682008368, 0.6317991631799164]
Logistic Regression Mean accuracy: 0.66
Decision Tree Accuracy for each fold: [0.5916666666666667, 0.5875, 0.5606694560669456, 0.6317991631799164, 0.5941422594142259]
Decision Tree Mean accuracy: 0.59
AdaBoost Accuracy for each fold: [0.5916666666666667, 0.6708333333333333, 0.6108786610878661, 0.6820083682008368, 0.6150627615062761]
AdaBoost Mean accuracy: 0.63
QDA Accuracy for each fold: [0.6083333333333333, 0.6416666666666667, 0.5983263598326359, 0.6317991631799164, 0.6192468619246861]
QDA Mean accuracy: 0.62


In [52]:
# Step 1: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=8)

X_train = X_train[300:]
y_train = y_train[300:]

# Step 2: Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 3: Build the neural network model
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),  # First hidden layer
    Dropout(0.3),  # Dropout for regularization
    Dense(32, activation='relu'),  # Second hidden layer
    Dense(1, activation='sigmoid')  # Output layer (binary classification)
])

# Step 4: Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 5: Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Step 6: Evaluate the model on the test set
y_pred = (model.predict(X_test) > 0.5).astype(int)  # Convert probabilities to binary predictions
accuracy = accuracy_score(y_test, y_pred)

print(f"Neural Network Test Accuracy: {accuracy:.2f}")


Epoch 1/100
17/17 [==============================] - 0s 5ms/step - loss: 0.6959 - accuracy: 0.5475 - val_loss: 0.6790 - val_accuracy: 0.5611
Epoch 2/100
17/17 [==============================] - 0s 2ms/step - loss: 0.6427 - accuracy: 0.6294 - val_loss: 0.6534 - val_accuracy: 0.6000
Epoch 3/100
17/17 [==============================] - 0s 2ms/step - loss: 0.6314 - accuracy: 0.6350 - val_loss: 0.6439 - val_accuracy: 0.6056
Epoch 4/100
17/17 [==============================] - 0s 2ms/step - loss: 0.6207 - accuracy: 0.6462 - val_loss: 0.6376 - val_accuracy: 0.6111
Epoch 5/100
17/17 [==============================] - 0s 2ms/step - loss: 0.5987 - accuracy: 0.6760 - val_loss: 0.6359 - val_accuracy: 0.6139
Epoch 6/100
17/17 [==============================] - 0s 2ms/step - loss: 0.6025 - accuracy: 0.6760 - val_loss: 0.6337 - val_accuracy: 0.6028
Epoch 7/100
17/17 [==============================] - 0s 2ms/step - loss: 0.5846 - accuracy: 0.6853 - val_loss: 0.6345 - val_accuracy: 0.6167
Epoch 8/100
1